_Main topics covered during today's session:_

Previous NB:

1. **Pandas Functions:**
    
    a. Index operations
    
    b. Concat
    
    c. Merge
    
    d. Groupby and Aggregation

This NB:

2. **Troubleshooting pandas dataframes**

*****************************************

# Troubleshooting pandas dataframes

*****************************************

## What we want to do is show some techniques for troubleshooting when our dataframe does not match the solution dataframe. 

*Steps for today:*

1. We have the who3 and who3_soln dataframes, from Notebook 7.

2.  We will make a copy of the who3 dataframe and manipulate it a little bit.

3.  Also run the function to show that they are equivalent, at the start.

#### First, let's import the pandas library and define the functions.

The two functions below, canonicalize_tibble() and tibbles_are_equivalent(), are from the Notebook 7 solution.

In [ ]:
import pandas as pd

### Remember that `Y` is in canonical order if it has the following properties.

1. The variables appear in sorted order by name, ascending from left to right.
2. The rows appear in lexicographically sorted order by variable, ascending from top to bottom.
3. The row labels (`Y.index`) go from 0 to `n-1`, where `n` is the number of observations.

In [ ]:
def canonicalize_tibble(X):
    # Enforce Property 1:
    var_names = sorted(X.columns)
    Y = X[var_names].copy()

    ### BEGIN SOLUTION
    # Enforce Property 2:
    Y.sort_values(by=var_names, inplace=True)
    
    # Enforce Property 3:
    Y.reset_index(drop=True, inplace=True)
#     Y.sort_index(inplace=True)
    ### END SOLUTION
    return Y

#### Below is the tibbles_are_equivalent() function. The testing function on the exams is similar to this one.

#### However, there is one key difference between the two functions. It is that the testing code uses the Python function assert_frame_equal() to do the comparison. We will show how this works, and how you can use it in your debugging, below.

Here is the link to the documentation page for the function:  https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.testing.assert_frame_equal.html


In [ ]:
def tibbles_are_equivalent(A, B):
    """Given two tidy tables ('tibbles'), returns True iff they are
    equivalent.
    """
    ### BEGIN SOLUTION
    A_hat = canonicalize_tibble(A)
    B_hat = canonicalize_tibble(B)
    equal = (A_hat == B_hat)
    return equal.all().all()
    ### END SOLUTION

In [ ]:
# bring in the two dataframes to work with
who3_soln = pd.read_csv('who3_soln.csv')
who3 = pd.read_csv('who3.csv')

#### Let's make a copy of our code output df to work with.

In [ ]:
who3_skillsOH = who3.copy()

### What are the checks that assert_all_equal() does?
 
1. Checks for data types in the two dfs. 
2. Checks the number of rows and columns in the two dfs. 
3. Checks the column names in the two dfs.
4. Finally, compares the actual data in the two dfs.

#### Let's look at each one individually, and how to troubleshoot.

#### You can use this Pandas function to do your testing and troubleshooting, and we provide the code below that you can copy and paste into your notebooks to troubleshoot.

In [ ]:
def pandas_troubleshoot(a,b):  # pass in the two dfs to compare as variables a and b
    from pandas.testing import assert_frame_equal
    
    def canonicalize_tibble(X, remove_index=True):
        var_names = sorted(X.columns)
        Y = X[var_names].copy()
        Y.sort_values(by=var_names, inplace=remove_index)
        Y.reset_index(drop=remove_index, inplace=remove_index)
        return Y
    
    a_hat = canonicalize_tibble(a)
    b_hat = canonicalize_tibble(b)
    try:
        assert_frame_equal(a_hat, b_hat, check_dtype=True,
                           check_index_type='equiv', check_column_type='equiv',
                           check_frame_type=True, check_names=True, 
                           by_blocks=False, check_exact=False, 
                           check_datetimelike_compat=False, 
                           check_categorical=True, check_like=False,
                           check_freq=True, check_flags=True,
                           rtol=1e-05, atol=1e-08, obj='DataFrame')
        print('True')
        return True
    except AssertionError as e:
        print(e)
        return e

### We will now set up 4 dataframes, that exercise the 4 types of tests that assert_all_equal() does. The changed dataframes will all be named for the type of change/test we are exercising.

#### What we will do is change each df and run some code that shows you how to do individual checks for this test.

#### Then we will show how you can do this using the assert_all_equal() function, and let it tell you what has failed.

#### First, we will change a datatype. 

https://www.geeksforgeeks.org/get-the-datatypes-of-columns-of-a-pandas-dataframe/

In [ ]:
who3_skillsOH_type = who3_skillsOH.copy()
who3_skillsOH_type["count"] = who3_skillsOH_type["count"].astype(float)  

In [ ]:
display(who3_skillsOH_type.dtypes)
display(who3_soln.dtypes)

#### Second, we change the number of rows in the dataframe.
https://www.geeksforgeeks.org/count-the-number-of-rows-and-columns-of-pandas-dataframe/

In [ ]:
who3_skillsOH_drop = who3_skillsOH.copy()
# Dropping last 10 rows using drop
who3_skillsOH_drop.drop(who3_skillsOH_drop.tail(10).index,inplace = True)

#note that printing the head() of the df does not tell you there is an error
print(who3_skillsOH_drop.head())
print(who3_soln.head())

In [ ]:
# # fetching the number of rows and columns
rows_OH = who3_skillsOH_drop.shape[0]
cols_OH = who3_skillsOH_drop.shape[1]

rows_soln = who3_soln.shape[0]
cols_soln = who3_soln.shape[1]
  
# displaying the number of rows and columns
print("Rows OH: " + str(rows_OH))
print("Columns OH: " + str(cols_OH))
print("Rows soln: " + str(rows_soln))
print("Columns soln: " + str(cols_soln))

#### Third, we will change a couple of column names.

In [ ]:
# # Change a couple of the column names in the new df, just to generate the error.
who3_skillsOH_colname = who3_skillsOH.copy()
who3_skillsOH_colname.rename(columns = {'country':'Country'}, inplace = True)
who3_skillsOH_colname.rename(columns = {'age_group':'age-group'}, inplace = True)

In [ ]:
my_list = who3_skillsOH_colname.columns.values.tolist()
soln_list = who3_soln.columns.values.tolist()

for i,col_name in enumerate(my_list):
    if col_name != soln_list[i]:
        print('Column names do not match')
        print('My column name: ', col_name)
        print('Soln column nm: ', soln_list[i])

#### Finally, let's change some values in the df.

In [ ]:
# # let's change a few values in our skillsOH dataframe
who3_skillsOH_data = who3_skillsOH.copy()
# # make some data changes
who3_skillsOH_data.at[0, 'count'] = 6
who3_skillsOH_data.at[1, 'count'] = 2
who3_skillsOH_data.at[2, 'year'] = 2011
who3_skillsOH_data.at[3, 'year'] = 2012

In [ ]:
# # note that the "self" df is the df that we are running compare on
# # and the "other" df is the one in the parenthesis (we are comparing to)
display(who3_soln.compare(who3_skillsOH_data))

# # align the differences on rows, just a different way of looking at the comparison
display(who3_soln.compare(who3_skillsOH_data,align_axis=0))

### Now let's use our pandas_troubleshoot() function to figure out what the error is.

In [ ]:
a,b = who3_skillsOH_type,who3_soln   # data type differences of columns
result = pandas_troubleshoot(a,b)
result

In [ ]:
a,b = who3_skillsOH_colname,who3_soln   # column names different
result = pandas_troubleshoot(a,b)
result

In [ ]:
a,b = who3_skillsOH_drop,who3_soln   # differences in the number of rows of data
result = pandas_troubleshoot(a,b)
result

In [ ]:
a,b = who3_skillsOH_data,who3_soln   # data differences
result = pandas_troubleshoot(a,b)
result

# Notice that we changed "count" and "year" above, but the error only returns for "count".
# The function returns the first column with errors, and you may need to run this multiple
# times, if you have multiple columns with data errors.